In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# There needs to be a folder located in ./ with folder name as declared in results_folder.
# Inside this folder there need to be these files:
# dataset_name + _results_opt.txt - The file containing the results for opt for each file for each value of d
# dataset_name + _lengths.txt - The file containing the length of each file in the dataset
# dataset_name + _results_algos.txt - The file containing the results for each algorithm for each value of d
results_folder = "./opt_results/"
dataset_name = "large"
ds = [1, 2, 3, 4, 5, 6, 10, 20, 50, 100]

## Theoretical c-competitiveness

In [ ]:
# This function does not need any input to calculate the values.
# It calculates the theoretical c-competitiveness values for algorithms CD, CND, RR and TS
# Output is a list (size 4) of lists, each of which has the c-competitiveness values for different
# values of d for the respective algorithm
def calc_c_competitiveness_theo():    
    # Counter Deterministic
    ls = [1, 2, 2, 3, 4, 5, 8, 16, 39, 78]
    cd_comp = []
    for i, d in enumerate(ds):
        c = max(3 + 2*ls[i]/d, 2+2*d/ls[i])
        cd_comp.append(round(c, 2))
        
    ls = [2, 5, 7, 10, 12, 15, 25, 51, 128, 256]
    cnd_comp = []
    for i, d in enumerate(ds):
        l = ls[i]
        c = max(1 + (l+1)/(2*d), 1 + (2*d + (l+1)/2)/l)
        cnd_comp.append(round(c, 2))

    ls = [3, 5, 8, 10, 13, 15, 26, 51, 128, 256]
    ps1 = [0.451, 0.210, 0.137, 0.101, 0.080, 1/15, 0.04, 0.02, 0.008, 0.004]
    psl = [0.0971, 0.16, 0.043, 0.089, 0.035, 1/15, 0.0095, 0.017, 0.005, 0.003]
    rr_comp = []
    for i, d in enumerate(ds):
        l = ls[i]
        p1 = ps1[i]
        pl = psl[i]
        c = max(1 + (((l-1)*l/2)*p1 + l*pl)/d, 1 + p1*(2*d + (l-1)*l/2*p1 + l*pl))
        rr_comp.append(round(c, 2))
        
    
    ts_comp = []
    ls = [1, 2, 4, 5, 6, 7, 12, 26, 60, 119]
    ps = [0.45793, 0.45798, 0.4582, 0.4579, 0.458, 0.45787, 0.38835, 0.5, 0.459, 0.4579]
    
    for i, d in enumerate(ds):
        l = ls[i]
        p = ps[i]
        c1 = 1 + (1/2 + max(1, 2*p)*(1-p))*l/d
        c2 = 1 + 3*p/2 - p*p + 2*p/(l/d)
        c3 = (7-3*p)/4 + 1/(l/d)
        c4 = (3 - p + p*p)/2 + 2*(1 - 2*p + p*p)/(l/d)
        c5 = (3 + p - p*p)/2 + 2*p*p/(l/d)
        c6 = 2 - p + (1-p)/(l/d)
        c = max(c1, c2, c3, c4, c5, c6)
        ts_comp.append(round(c, 2))
    
    theo_comp = [cd_comp, cnd_comp, rr_comp, ts_comp]
    return theo_comp

## Experimental c-competitiveness

In [ ]:
# This function calculates the experimental competitiveness ratio 
# of the online algorithms.
# Input: algos_cost: a list (of size 4) of lists containing the cost of each online algorithm
#                    for all values of d
#        opt_cost: a list containing the cost of the optimal algorithm for all values of d
# Output: a list (of size 4) of lists containing the experimental c-competitiveness of
#         each of the online algorithms for all values of d
def calc_c_competitiveness_exp(algos_cost, opt_cost):
    cd_comp_exp = []
    cnd_comp_exp = []
    rr_comp_exp = []
    ts_comp_exp = []
    for i, d in enumerate(ds):
        cd_comp_exp.append(round(algos_cost[i][0]/opt_cost[i], 2))
        cnd_comp_exp.append(round(algos_cost[i][1]/opt_cost[i], 2))
        rr_comp_exp.append(round(algos_cost[i][2]/opt_cost[i], 2))
        ts_comp_exp.append(round(algos_cost[i][3]/opt_cost[i], 2))
        
    exp_comp = [cd_comp_exp, cnd_comp_exp, rr_comp_exp, ts_comp_exp]
    
    return exp_comp

## c-competitiveness errors

In [ ]:
# Calculate the relative errors between the theoretical c-competitiveness ratios
# and the experimental c-competitiveness.
# Input: theo_comp, exp_comp - a list (of size 4) of lists containing the theoretical (experimental)
#                              competitiveness ratio for each of the online algorithms for various
#                              values of d
def calculate_c_errors(theo_comp, exp_comp):
    relative_errors = []
    for i, theo_comp_i in enumerate(theo_comp):
        exp_comp_i = exp_comp[i]
        relative_errors.append(np.round((np.array(theo_comp_i) - np.array(exp_comp_i))/np.array(theo_comp_i), 2))
    return relative_errors

## Calculate cost of opt

In [ ]:
# Reads from a file located in dataset_path and parses and calculates the costs 
# of the optimal offline algorithm. The file from which it reads has the costs of
# all the files of the dataset for all values of d. The values for each of the files
# in the corpus are averaged.
# Input: dataset_path - the file from which to read the costs
#        lengths_path - a file where the lengths of each of the files
#                       in the corpus is found.
# Output: average_opt_cost - the optimal cost to serve all files in the corpus
#                            averaged over all files
def calculate_opt_cost(dataset_path, lengths_path):
    
    #Read the approx opt cost
    opt_file = open(dataset_path, 'r').read()
    
    #Read the lengths of files
    lengths = [int(line.rstrip('\n')) for line in open(lengths_path)]
    lengths_sum = sum(lengths)
    
    all_files_costs = []
    for d in ds:
        regex = str(d) + "\t(\d*)"
        matchobj = re.findall(regex, opt_file, flags=0)
        all_files_costs.append(matchobj)

    average_opt_cost = [(sum(np.array(d_opt).astype(np.float)) + lengths_sum)/len(d_opt) for d_opt in all_files_costs]
    # To be commented for other datasets rather than Canterbury. 
#     average_opt_cost[8] = (sum(np.array(all_files_costs[8]).astype(np.float)) + lengths_sum - lengths[7])/len(all_files_costs[8]) 
#     average_opt_cost[9] = (sum(np.array(all_files_costs[9]).astype(np.float)) + lengths_sum - lengths[7])/len(all_files_costs[9]) 
    return average_opt_cost

## Calculate cost of online algorithms

In [ ]:
# This function calculates the costs of all the online algorithms for the given dataset
# Input: dataset_path - the path to the file where the costs of the online algorithms
#                       can be found. It should be a 10 x 4 matrix of costs. 
# Output: costs - a list (of size 4) of lists, each of which contains the cost of the
#                 online algorithms for various sizes of d
def calculate_algos_cost(dataset_path):
    costs = [[float(cost) for cost in line.strip('\n').split('\t')] for line in open(dataset_path)]
    return costs

### Utility plotting function

In [ ]:
# Plots the costs.
# Input: all_costs - a list (of size 4) of lists each of which contains the costs
#                    of the online algorithms for various values of d
#        opt_cost - a list with the costs of the optimal offline algorithm for 
#                   various values of d
def plot_costs(all_costs, opt_cost, path_to_save):
    cd_cost = [item[0] for item in all_costs]
    cnd_cost = [item[1] for item in all_costs]
    rr_cost = [item[2] for item in all_costs]
    ts_cost = [item[3] for item in all_costs]
    
    ind = np.arange(len(cd_cost))  # the x locations for the groups
    width = 0.15  # the width of the bars

    fig, ax = plt.subplots(nrows=1, ncols=1)
    
    # plt.subplot(1, 2, 1)
#     rects1 = ax.bar(ind - 3*width/2, cd_cost, width, label='CD', color = 'black')
#     rects2 = ax.bar(ind - width/2, cnd_cost, width, label='CND', color = 'dimgray')
#     rects3 = ax.bar(ind + width/2, rr_cost, width, label='RR', color = 'silver')
#     rects4 = ax.bar(ind + 3*width/2, ts_cost, width, label='TS', color = 'white', edgecolor='black')
#     rects5 = ax.bar(ind + 5*width/2, opt_cost, width, label='OPT', color = 'white', edgecolor='black', hatch='/')
    rects1 = ax.bar(ind - 3*width/2, cd_cost, width, label='CD')
    rects2 = ax.bar(ind - width/2, cnd_cost, width, label='CND')
    rects3 = ax.bar(ind + width/2, rr_cost, width, label='RR')
    rects4 = ax.bar(ind + 3*width/2, ts_cost, width, label='TS')
    rects5 = ax.bar(ind + 5*width/2, opt_cost, width, label='OPT')
    ax.legend(loc='upper left', bbox_to_anchor=(1,1))

    ax.set_xticks(ind)
    ax.set_xticklabels(("1", "2", "3", "4", "5", "6", "10", "20", "50", "100"))
    ax.set_xlabel("d")
    ax.set_ylabel("Total cost")

    fig.tight_layout()
    fig.subplots_adjust(top=0.83)
    plt.savefig(path_to_save)
    plt.show()

In [ ]:
opt_results_path = results_folder + dataset_name + "_results_opt.txt"
lengths_path = results_folder + dataset_name + "_lengths.txt"
algos_results_path = results_folder + dataset_name + "_results_algos.txt"

opt_cost = calculate_opt_cost(opt_results_path, lengths_path)
algos_cost = calculate_algos_cost(algos_results_path)

In [ ]:
save_path = results_folder + dataset_name + "_costs_plot.png"
plot_costs(algos_cost, opt_cost, save_path)

In [ ]:
exp_comp = calc_c_competitiveness_exp(algos_cost, opt_cost)
theo_comp = calc_c_competitiveness_theo()
relative_errors = calculate_c_errors(theo_comp, exp_comp)

In [ ]:
for i, d in enumerate(ds):
    print(d, "&", theo_comp[0][i], '&', exp_comp[0][i], '&', round(relative_errors[0][i], 2), "\n",
             "&", theo_comp[1][i], '&', exp_comp[1][i], '&', round(relative_errors[1][i], 2), "\n"
             "&", theo_comp[2][i], '&', exp_comp[2][i], '&', round(relative_errors[2][i], 2), "\n"
             "&", theo_comp[3][i], '&', exp_comp[3][i], '&', round(relative_errors[3][i], 2), "\\\ \hline", "\n")